In [ ]:
import cProfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import os
import math
import json
import scipy
import sys
import time
from multiprocessing import Process, Manager
from operator import itemgetter
import pymp

# this inserts the continuous file into the path
sys.path.insert(0, '../Simulated Annealing/Continuous')
#from Anneal_cont import Annealer
import multiprocessing as mp
%matplotlib inline

pd.options.display.float_format = '{:,.1f}'.format
# sns.set()
# DIMS=(16, 6)
# FIGSIZE = (16, 5)

import operator

In [ ]:
def sinusoid_ladder(h):
    
    """
    A 1D sloped sinusoidal function with a decreasing gradient with increasing coordinate, defined here on the range -15 to 15.
    The parameter controls the height of the sine curves: as h --> inf, their height (the size of the peaks) grows,
    and as h --> 0, the curve approaches a straight line.
    """
    xrange = np.linspace(-15,15,1000,endpoint = True)
    
    # avoid NaN if input is h = 0
    if h == 0:
        h += 1e-6

    def func(x):
        return np.cos(x - 10) - (1/h)*x 
    
    return func, xrange

###### TWO-DIMENSIONAL FUNCTIONS ######

def levy():
    
    """
    Levy function N. 13
    See https://www.sfu.ca/~ssurjano/levy13.html
    
    The function is usually evaluated on the square x,y ∈ [-10, 10].
    The minimum is located at (x,y) = (1,1), and has value 0.
    """
    
    xrange = np.linspace(-10,10,1000,endpoint = True)
    yrange = np.linspace(-10,10,1000,endpoint = True)
    
    def func(x,y):  
        return np.sin(3 * np.pi * x) ** 2 + \
              (x - 1) ** 2 * (1+ np.sin(3 * np.pi * y) ** 2) + \
              (y - 1) ** 2 * (1+ np.sin(2 * np.pi * y) ** 2)
          
    return func, (xrange,yrange)

def bukin():

    """
    Bukin function N. 6 
    See https://www.sfu.ca/~ssurjano/bukin6.html
    
    The function is usually evaluated on the rectangle x ∈ [-15, -5], y ∈ [-3, 3]. 
    The sixth Bukin function has many local minima, all of which lie in a ridge.
    The minimum is located at (x,y) = (-10,1), and has value 0.
    """
    
    xrange = np.linspace(-15, -5, 1000, endpoint = True)
    yrange = np.linspace(-3,3,1000,endpoint = True)

    def func(x,y):
        return 100 * np.sqrt(abs(y - 0.01*x**2)) + 0.01 * abs(x + 10)
    
    return func, (xrange,yrange)

def easom():

    """
    Easom function
    See https://www.sfu.ca/~ssurjano/easom.html
    
    The function is usually evaluated on the square x, y ∈ [-100, 100].
    The Easom function has several local minima. It is unimodal, and the global minimum has a small area relative to the search space. 
    The global minimum is at (x,y) = (pi,pi), and has value -1
    """
    
    xrange = np.linspace(-100,100,1000,endpoint = True)
    yrange = np.linspace(-100,100,1000,endpoint = True)

    def func(x,y):
        return -np.cos(x)*np.cos(y)*np.exp( -(x - np.pi)**2 - (y - np.pi)**2 )
    
    return func, (xrange,yrange)

###### MULTI-DIMENSIONAL FUNCTIONS ######

def rastrigin(dim):
    
    """
    Rastrigin function
    See https://www.sfu.ca/~ssurjano/rastr.html
    
    The function is usually evaluated on the hypercube xi ∈ [-5.12, 5.12], for all i = 1, …, d.
    The Rastrigin function has several local minima. It is highly multimodal, but locations of the minima are regularly distributed.
    The global minimum is at (x,y,...) = (0,0,...,0), and has value 0
    
    Input: dim (desired dimension)    
    Returns: function values and the associated meshgrid (useful for plotting if dim=2)
    """
    
    xrange = np.linspace(-5.12,5.12,1000,endpoint = True)
    args = [xrange]*dim
    nd_mesh = np.meshgrid(*args)

    def func(nd_mesh):
        return 10*dim + sum( [nd_mesh[i]**2 - 10*np.cos(2*np.pi*nd_mesh[i]) for i in range(dim) ] )
        
    return func, nd_mesh

def michaelwicz(dim):
    
    """
    Michaelwicz function
    See https://www.sfu.ca/~ssurjano/michal.html
    
    The Michalewicz function has d! local minima, and it is multimodal. The parameter m defines 
    the steepness of they valleys and ridges; a larger m leads to a more difficult search. 
    The recommended value of m is m = 10.
    The function is usually evaluated on the hypercube xi ∈ [0, π], for all i = 1, …, d. 
    
    Known global minima:
        
        for dim = 2, at (x,y) = (2.20, 1.57) with value -1.8013
        for dim = 5, with value -4.687658
        for dim = 10, with value -9.66015 
        
    Input: dim (desired dimension) 
    Returns: function values and the associated meshgrid (useful for plotting if dim=2)
    """
    
    m = 10 # A free parameter recommended (on the website above) to be 10.
    
    xrange = np.linspace(0,np.pi,1000,endpoint = True)
    args = [xrange]*dim
    nd_mesh = np.meshgrid(*args)
    
    def func(nd_mesh):
        return -1*sum( [ np.sin(nd_mesh[i]) * np.sin(i*nd_mesh[i]/np.pi)**(2*m) for i in range(dim) ] )
    
    return func, nd_mesh

In [ ]:
func, mesh = levy()
i1, i2 = mesh[0], mesh[1]
def f():
    '''
    input: tour (list)

    Function that evaluates the cost of a given x1, x2 (euclidean distance)

    output: single cost
    '''
        
    exploration_space = [(i,j) for i in i1 for j in i2]
        # storing this huge dictionary in memory may not be a great idea..
    super_energies = {(i,j):[func(*[i,j]),0] for i,j in exploration_space}
    return super_energies

super_energies = f()

In [11]:
class ParallelTemp:

    '''
    Pass the max steps you want to take to the annealer function
    '''

    def __init__(
          self,
        maxsteps=500,
        explore=30,
        walkers=10,
        error_thres=10e-2, 
      #accs = [500, 1, 1, 0.5, 0, round((Ncity.n)**0.5), 30]
        swapping_choice="neighbor",
          ):
        '''
        inputs: total number of steps to try, geometric multiplier for annealing schedule
        Initialize parameters
        output: none
        '''
        self.func, self.mesh = func, mesh
        
        self.Tmax, self.exploration_space = maxsteps, explore
        self.explore = int(2e2)
        
        self.i1, self.i2 = self.mesh[0], self.mesh[1]
        self.all_energies = super_energies.copy()
        self.correct_answer, self.error_threshold, self.cumulative_correct = super_energies[min(self.all_energies.keys(), key=(lambda k: self.all_energies[k]))][0], error_thres, 0.0
        self.walkers_t1, self.walkers_t2, self.initial = walkers, walkers, walkers
        
        # partition function could possibly be used as a possible swapping mechanism
        self.stat_weight_ratio = dict()
        self.partition_function = 0
        
        # e_diff is a lambda function used to calculate the ratio of statistical weight
        self.e_diff = lambda x, y: np.exp(-(x[1] - x[0]) * y) 
#         self.anneal()
    
    def select_swap(self, walker_pos):
        # choose 2 walkers at random and return them from the list of keys
        keys = [i[0] for i in walker_pos]
        proposed_walker = [random.choice(keys) for _ in range(2)]
        return proposed_walker
    
    def random_neighbour(self):
        """ 
        input: x (a 2D array)
        
        draw from the entire array space of x1 and x2
        
        output: (newx, newy)
        """

        new_x = np.random.choice(self.i1)
        new_y = np.random.choice(self.i2)

        return [new_x, new_y]
    
    def f(self):
        '''
        input: tour (list)

        Function that evaluates the cost of a given x1, x2 (euclidean distance)

        output: single cost
        '''
        
        exploration_space = [(i,j) for i in self.i1 for j in self.i2]
        # storing this huge dictionary in memory may not be a great idea..
        super_energies = {(i,j):[self.func(*[i,j]),0] for i,j in exploration_space}
        return super_energies

    def swap_acceptance(self, t0, t1, walker_pos, p1, p2):
        '''
        this function takes in the current Beta and previous Beta, the two walkers proposed for swap and calculates
        the acceptance probabbility of the two walkers
        '''
        exp_function = np.exp((t1-t0)*(walker_pos[p1][1] - walker_pos[p0][1]))
        assert exp_function == 1
        return min(1, exp_function)
    
    
    def acceptance_probability(
        self,
        cost,
        new_cost,
        temperature,
        ):
        '''
        inputs: old cost, new cost, current temperature

        calculate probability of acceptance and return it using the metropolis algorithm

        output: probability (0 to 1)
        '''

        return np.exp(-(new_cost - cost) / temperature)
    
    def check_correct(self, energy):
        self.cumulative_correct += np.sum([1 if (i-self.correct_answer)<=self.error_threshold or i<self.correct_answer else 0 for i in energy])
    
#     def max_key(self, walker_pos):
#         '''
#         inputs: none
#         finds the minimum value in the dictionary of walkers
#         outputs: key of the lowest (best) cost value in the entire dictionary of walkers
#         '''
#         return min(walker_pos.keys(), key=(lambda k: walker_pos[k][1]))
    
    def max_val(self, walker_pos, i=2):
        return max(enumerate(map(itemgetter(i), walker_pos)),key=itemgetter(1))

    
    def anneal(self):
        '''
        inputs: none

        function performs annealing and calls random start to kickstart the annealing process. iteratively
        calculates the new cost.

        output: final cost, final state (list of x1 and x2), all costs (list of costs at every timestep)

        '''
        T_list = [i for i in range(self.Tmax)]
        # metrics we want to keep track of
        free_energy = dict()
        average_cost = list()
        best = list()
        walker_z = list()
        self.walker_pos, self.new_walker_pos = list(), list()
        proposed_swaps = dict()
        temp_walker = dict()
        
        configs_explored = dict()
        # generate a state of random walkers with their costs, need to change such that we are generating tours instead of other stuff.

        # generate a state of random walkers at each temperature, with their costs
        self.walker_pos = [[i,[np.random.choice(self.i1),
                             np.random.choice(self.i2)]] for i in range(1,self.Tmax+1)]
        
        # gives the walker at each tempearture
        temp_walker = {i[0]:round(1/i[0], 2) for i in self.walker_pos}
        
        # append the cost of each state 
        for i,j in enumerate(self.walker_pos):
            self.walker_pos[i].append(self.all_energies[tuple(j[1])][0])
            
            # increase the number of walkers at all_energies
            self.all_energies[tuple(j[1])][1] += 1
        
        # gets the maximum value of the key 
        # rewrite this
        max_key = self.max_val(self.walker_pos)[0]
        
        best_cost = [[1, self.walker_pos[max_key][1], self.walker_pos[max_key][2]]]

        # run the monte carlo sweeps in parallel
        start_time = time.time()            
        L = pymp.shared.list([[i[0],i[1],i[2]] for i in self.walker_pos])   
#         L = pymp.shared.array((self.Tmax+1,len(self.walker_pos[0])), dtype='uint8')        
        with pymp.Parallel(4) as p:
            
            for temp_step in range(1, self.explore):
                print("Temperature: {}".format(temp_step))
                
                for index in p.range(len(self.walker_pos)):
                        
                    # mcmc sweep
                    T = temp_walker[self.walker_pos[index][0]]
                    
                    costs = round(self.walker_pos[index][2], 2)
                    states = self.walker_pos[index][1]

                    for step in range(self.exploration_space):
                        new_state = self.random_neighbour()
                        new_cost = func(*new_state)

                    if new_cost < costs or self.acceptance_probability(costs,
                          new_cost, T) >= random.uniform(0, 1):
                        states, costs = new_state, new_cost

                    # walker
                    L[index][0] = self.walker_pos[index][0]
                    # position
                    L[index][1] = states
                    # cost
                    L[index][2] = costs
                
                self.new_walker_pos = L                    
    
                max_key = self.max_val(self.new_walker_pos)[0]

                best_cost.append([temp_walker[max_key], self.new_walker_pos[max_key][0], self.walker_pos[max_key][1]])

                swap1, swap2 = self.select_swap(self.walker_pos)

                # swap both the position, cost, and temperatures
                self.walker_pos[swap1], self.walker_pos[swap2] = self.walker_pos[swap2], self.walker_pos[swap1]
                temp_walker[swap1], temp_walker[swap2] = temp_walker[swap2], temp_walker[swap1]


                all_costs = np.array([i[2] for i in self.walker_pos])

                average_cost.append(np.mean(all_costs))

                self.check_correct(all_costs/self.initial)

                # only after you are done calculating the covariance, then you reassign the previous to the currxent one
                
                
                self.walker_pos = list(self.new_walker_pos) if len(list(self.new_walker_pos)) > 0 else 
                    

#                 L.clear()
#             pool.close()
        
        print("Time Taken (Multiprocessing): {}s".format(time.time() - start_time))
        
        return (
                average_cost,
                self.cumulative_correct,
                best_cost,
                best,
                T_list, 
                )

SyntaxError: invalid syntax (<ipython-input-11-cd448005a877>, line 211)

In [12]:
a = ParallelTemp()
average_cost, converged, best_cost, best, temperature = a.anneal()


Temperature: 1
Temperature: 1
Temperature: 1


/Users/tinkertanker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/Users/tinkertanker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars


Temperature: 1


/Users/tinkertanker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars


Temperature: 2
Temperature: 2
Temperature: 2
Temperature: 2
Temperature: 3
Temperature: 3
Temperature: 3
Temperature: 3
Temperature: 4
Temperature: 4
Temperature: 4
Temperature: 4
Temperature: 5
Temperature: 5
Temperature: 5
Temperature: 5


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

EOFError: 

## Sample

In [ ]:
import multiprocessing
import ctypes
import numpy as np

walkers_t1 = 10
func, mesh = levy()
i1, i2 = mesh[0], mesh[1]
all_energies = super_energies.copy()
acceptrate, lams, multiplier = 0.5, 0, 1
Tmax, exploration_space = 1000, 10

def random_neighbour():
    """ 
    input: x (a 2D array)
        
    draw from the entire array space of x1 and x2
        
    output: (newx, newy)
    """

    new_x = np.random.choice(i1)
    new_y = np.random.choice(i2)

    return [new_x, new_y]

def f():
    '''
    input: tour (list)

    Function that evaluates the cost of a given x1, x2 (euclidean distance)

    output: single cost
    '''

    exploration_space = [(i,j) for i in i1 for j in i2]
    # storing this huge dictionary in memory may not be a great idea..
    super_energies = {(i,j):[func(*[i,j]),0] for i,j in exploration_space}
    return super_energies

def acceptance_probability(
    cost,
    new_cost,
    temperature,
    ):
    '''
    inputs: old cost, new cost, current temperature

    calculate probability of acceptance and return it using the metropolis algorithm

    output: probability (0 to 1)
    '''

    return np.exp(-(new_cost - cost) / temperature)

def anneal(T, L, k, v):
    '''
    inputs: none

    function performs annealing and calls random start to kickstart the annealing process. iteratively
    calculates the new cost.

    output: final cost, final state (list of x1 and x2), all costs (list of costs at every timestep)

    '''

    # this is where the multiprocessing can start from

    costs = round(walker_pos[k][1], 2)
    states = walker_pos[k][0]

    walker_pos_check = walker_pos.copy()
    for step in range(exploration_space):
        new_state = random_neighbour()
        new_cost = func(*new_state)
    
    if new_cost < costs or acceptance_probability(costs,
                      new_cost, T) >= random.uniform(0, 1):
        states, costs = new_state, new_cost
    
    if lams == 1:
        acceptrate = 1 / 500 * (499 * acceptrate + 1)
    else:

        if lams == 1:

            acceptrate = 1 / 500 * (499 * acceptrate)

            # check conditions

            if fraction < 0.15:
                LamRate = 0.44 + 0.56 * 560 ** (-temp_step
                  / (Tmax * 0.15))
            elif fraction < 0.65:
                LamRate = 0.44
            else:
                LamRate = 0.44 * 440 ** ((-fraction - 0.65) / 0.35)

            if LamRate < acceptrate:
                T *= 0.99
            else:
                T *= 1 / 0.999

    L[k] = [states, costs]
#     walker_pos[k][0], walker_pos[k][1] = states, costs
#     L = {k:v for k,v in walker_pos.items()}


if __name__ == '__main__':
    T_list = [1]    
    populations = list()
    free_energy = dict()
    average_cost = list()
    best = list()
    walker_z = list()
    walker_pos, new_walker_pos = dict(), dict()

    configs_explored = dict()
    taus_over_time = dict()

    walker_pos = {i:[[np.random.choice(i1),
                         np.random.choice(i2)]] for i in range(walkers_t1)}        

    # append the cost of each state 
    for k,v in walker_pos.items():
        walker_pos[k].append(all_energies[tuple(v[0])][0])

        # increase the number of walkers at all_energies
        all_energies[tuple(v[0])][1] += 1
    
    start_time = time.time()
    with Manager() as manager:
        L = manager.dict()  # <-- can be shared between processes.
        with manager.Pool(processes=4) as pool:
            
            for temp_step in range(2, Tmax+2):
                print(temp_step)
                #covariance_matrix = np.zeroes([self.walkers_t1, self.walkers_t1])
                # calculate the temperature from temp step 2 onward

                fraction = 1/temp_step

                if temp_step > 2:
                    T = multiplier * fraction if multiplier < 1 else fraction
                else:
                    T = fraction 

                T_list.append(int(np.round(1/T)))

                populations.append(walkers_t1)


                # multiprocessing starts here #

                
                for k,v in walker_pos.items():
                    pool.apply_async(anneal, args=(T, L, k, v))

    #             pool = multiprocessing.Pool(processes=3)
    #             #pool.map(generate_point, range(walkers_t1))
    #             jobs = []

    #             print(walker_pos)
    #             for k,v in walker_pos.items():
    #                 p = multiprocessing.Process(target=anneal, args=(T, L, k, v))
    #                 jobs.append(p)
    #                 p.start()

    #             pool.close()

    #             for proc in jobs:
    #                 proc.join()

                walker_pos = L
                
                #print(walker_pos)
            L.clear()
        pool.close()
            #print(L)
    end_time = time.time()
    print("Time Taken (Multiprocessing): {}s".format(end_time - start_time))


